In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:

def get_stock_dataframe(ticker: str, num_years: int = 10, interpolate_method='pad'):
    s = pd.read_csv(f'stocks/{ticker}.us.txt').set_index('Date')
    s.index = pd.to_datetime(s.index) # convert index to DateTime
    filled = s.asfreq(pd.offsets.BDay(), method=interpolate_method) # deal with missing dates by interpolating

    # adding new columns
    filled['Return'] = (filled['Close'] - filled['Close'].shift(1)) / filled['Close'] * 100
    filled['Close_7d_avg'] = filled['Close'].rolling(window=7).mean()
    filled['Close_30d_avg'] = filled['Close'].rolling(window=30).mean()
    filled['Volatility_30d'] = filled['Close'].rolling(window=30).std()
    
    # we have to keep last 10 years of data. I'll do this by going back 10 years from the last index value
    # this gives us a range from 2007 to 2017
    start_date = filled.index[-1] - pd.DateOffset(years=num_years)
    filtered = filled.loc[start_date:].copy()
    return filtered

def get_multi_stock_data(tickers, num_years: int = 10, interpolate_method='pad'):
    stocks = [get_stock_dataframe(t, num_years, interpolate_method) for t in tickers]
    return pd.concat(stocks, keys=tickers)
    

tickers = ['aapl', 'amzn', 'googl', 'msft', 'nvda']
stocks = get_multi_stock_data(tickers, 10, 'pad')
stocks

Open     High      Low    Close     Volume  OpenInt  \
     Date                                                                 
aapl 2007-11-12   21.130   21.479   19.291   19.691  492362604        0   
     2007-11-13   20.615   21.897   19.691   21.765  484373501        0   
     2007-11-14   22.733   22.739   20.970   21.274  403585172        0   
     2007-11-15   21.280   21.717   20.528   21.040  414487458        0   
     2007-11-16   21.193   21.388   20.405   21.309  385660112        0   
...                  ...      ...      ...      ...        ...      ...   
nvda 2017-11-06  207.200  209.980  206.700  209.630    9731783        0   
     2017-11-07  210.550  212.900  210.056  212.000   10671815        0   
     2017-11-08  211.850  212.000  207.240  209.160   13033902        0   
     2017-11-09  205.270  206.330  200.370  205.320   23895006        0   
     2017-11-10  213.080  218.670  211.630  216.140   31300857        0   

                   Return  Close_7d_avg  Close_30d_avg  Volatility_30d  
     Date                                                               
aapl 2007-11-12 -7.556752     22.808286      22.323267        1.479132  
     2007-11-13  9.529060     22.480571      22.372367        1.433111  
     2007-11-14 -2.307982     22.113571      22.407400        1.390984  
     2007-11-15 -1.112167     21.610714      22.441800        1.341479  
     2007-11-16  1.262377     21.246714      22.462933        1.317465  
...                   ...           ...            ...             ...  
nvda 2017-11-06  0.448409    206.281429     192.363667       11.094327  
     2017-11-07  1.117925    207.730000     193.698333       10.962750  
     2017-11-08 -1.357812    208.490000     194.812667       10.770689  
     2017-11-09 -1.870251    208.277143     195.800667       10.304477  
     2017-11-10  5.006015    209.554286     197.046333       10.432674  

[13050 rows x 10 columns]

In [ ]:
# Time for EDA stuff
# to find stock with highest average return
stocks.groupby(level=0).mean()['Return']
# AMZN wins!!

aapl     0.061652
amzn     0.073768
googl    0.027608
msft     0.026868
nvda     0.029568
Name: Return, dtype: float64

In [19]:
# finding the most volatile month for each stock
for t in tickers:
    print(t)
    s = stocks.loc[(t,)].copy()
    print(s.groupby(s.index.month)['Volatility_30d'].mean().sort_values(ascending=False).head(1).index)

aapl
Index([2], dtype='int32', name='Date')
amzn
Index([11], dtype='int32', name='Date')
googl
Index([11], dtype='int32', name='Date')
msft
Index([11], dtype='int32', name='Date')
nvda
Index([6], dtype='int32', name='Date')
